In [ ]:
%matplotlib inline
import quandl as Quandl
Quandl.ApiConfig.api_key = "_ktbP7wTMU7jSPHGDae7"
import numpy as np
import pandas as pd
import math
from sklearn import *
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style
import datetime
from hmmlearn import hmm

In [ ]:
gl = pd.read_csv("google.csv")

In [ ]:
#setting parameters
num_days = 365
k = 50
num_iters = 10000
# Possible number of states in Markov Model
state_range_min = 2
state_range_max = 15
state_range = range(state_range_min, state_range_max)

In [ ]:
# defining helper methods

# Calculate Mean Absolute Percentage Error of predictions
def calc_mape(forecast, actual):
    return np.divide(np.sum(np.divide(np.absolute(forecast - actual), actual), 0), actual.shape[0])

# Calculate the Bayesian Information Criterion
def calc_bic(num_params, score, n):
    return num_params * np.log(n) - 2 * score

# Find the optimal states in state_range with respect to the BIC score.
def find_opt_states(dataset, num_ters=1000):
    #Note, since the EM algorithm is a gradient-based optimization method, 
    #it will generally get stuck in local optima. 
    #You should in general try to run fit with various initializations and select the highest scored model.
    bic_vect = np.empty([0,1])
    for states in state_range: #try all different configurations of states
        model = hmm.GaussianHMM(n_components=states, covariance_type='full', tol=0.0001, n_iter=num_iters)
        model.fit(dataset.iloc[num_days:,:]) #fit using historical data. All but the most recent 365 days
        current_score = model.score(dataset)
        num_params = states**2 + states 
        bic_vect = np.vstack((bic_vect, calc_bic(num_params, current_score, dataset.shape[0])))
    opt_states = np.argmin(bic_vect) + state_range_min
    return opt_states

def predict(dataset, opt_states, num_days, k, num_iters):
    predicted_stock_data = np.empty([0,dataset.shape[1]])
    for idx in reversed(range(num_days)):
        train_dataset = dataset.iloc[idx + 1:,:]
        test_data = dataset.iloc[idx,:]; 
        num_examples = train_dataset.shape[0]
        if idx == num_days - 1:
            # If it's the first time around, use stmc to initialize.
            model = hmm.GaussianHMM(n_components=opt_states, covariance_type='full', tol=0.0001, n_iter=num_iters, init_params='stmc')
        else:
            # Retune the model by using the HMM paramters from the previous iterations as the prior
            model = hmm.GaussianHMM(n_components=opt_states, covariance_type='full', tol=0.0001, n_iter=num_iters, init_params='')
            model.transmat_ = transmat_retune_prior 
            model.startprob_ = startprob_retune_prior
            model.means_ = means_retune_prior
            model.covars_ = covars_retune_prior

        model.fit(np.flipud(train_dataset))

        transmat_retune_prior = model.transmat_
        startprob_retune_prior = model.startprob_
        means_retune_prior = model.means_
        covars_retune_prior = model.covars_
        iters = 1;
        past_likelihood = []
        curr_likelihood = model.score(np.flipud(train_dataset.iloc[0:k - 1, :]))
        while iters < num_examples / k - 1:
            past_likelihood = np.append(past_likelihood, model.score(np.flipud(train_dataset.iloc[iters:iters + k - 1, :])))
            iters = iters + 1
        likelihood_diff_idx = np.argmin(np.absolute(past_likelihood - curr_likelihood))
        predicted_change = train_dataset.iloc[likelihood_diff_idx,:] - train_dataset.iloc[likelihood_diff_idx + 1,:]
        predicted_stock_data = np.vstack((predicted_stock_data, dataset.iloc[idx + 1,:] + predicted_change))
        return predicted_stock_data

In [ ]:
dataset = gl[['Adj. Open','Adj. High', 'Adj. Low', 'Adj. Close']].copy()
dataset = dataset.iloc[::-1] #reverse the dataframe so we have the recent data at the top.
opt_state = find_opt_states(dataset)
dataset = gl[['Adj. Open','Adj. High', 'Adj. Low', 'Adj. Close']].copy()
dataset = dataset.iloc[::-1] #reverse the dataframe so we have the recent data at the top.
predicted_stock_data = predict(dataset, opt_state, num_days, k, num_iters)

In [ ]:
mape = calc_mape(predicted_stock_data, np.flipud(dataset.iloc[range(num_days),:]))
print('MAPE is ',mape)

In [ ]:
plt.plot(np.flipud((dataset[:365].reset_index()['Adj. Close'])), color = 'salmon', alpha = 0.5, label='Actual')
plt.plot(predicted_stock_data[:,3], color = 'lightblue', alpha = 0.5, label='Forecast')
plt.xlabel('Days')
plt.ylabel('Adj. Close')
plt.title('Google adjusted close price for 365 days')
plt.legend()
plt.savefig('adjustclose.png',dpi=300)